# Урок 3. Классификация. Логистическая регрессия.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log. 

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_logloss(y, y_pred):
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
  return err

In [4]:
def calc_logloss_modern(y, y_pred):
    err = np.zeros(y.shape)
    err[y==0] = -(1.0-y[y==0]) * np.log(1.0-y_pred[y==0])
    err[y==1] = -y[y==1] * np.log(y_pred[y==1])
    err = np.mean(err)    
    return err

In [5]:
# Пример применения
y1 = np.array([1, 0])
y1_pred = np.array([0.9, 0.1])
calc_logloss_modern(y1, y1_pred)

0.10536051565782628

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [6]:
def calc_std_feat(x):
  res = (x - x.mean()) / x.std()
  return res

In [7]:
def sigmoid(z):
  res = 1 / (1 + np.exp(-z))
  return res

In [51]:
def eval_model(X, y, iterations, alpha=1e-4):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    #if i % (iterations / 10) == 0:
     # print(i, W, err)
  return W,err

In [9]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [10]:
W,err = eval_model(X_st, y, iterations=1000, alpha=1e-4)
print(W)
print(err)

[ 0.45886878 -0.25439917  0.64531344  1.46695851]
1.0395158244739489


In [11]:
def params_to_find():
    iterations_to_find=0
    alpha_to_find=0
    W_to_find=0
    err_min=np.inf
    for iterations in np.arange( 1000,10000,500):
        for alpha in np.logspace(-5.0,-1.0,num=10):
            W,err = eval_model(X_st, y, iterations=iterations, alpha=alpha)
            if err < err_min:
                err_min=err
                alpha_to_find=alpha
                iterations_to_find=iterations
                W_to_find=W
                
        
    return err_min,alpha_to_find,iterations_to_find,W_to_find

In [12]:
err_min, alpha_to_find, iterations_to_find, W_to_find=params_to_find()
print("err min =",err_min)
print("alpha for err min =",alpha_to_find)
print("iterations for err min =",iterations_to_find)
print("W for err min =",W_to_find)


err min = 0.2554502533028652
alpha for err min = 0.1
iterations for err min = 9500
W for err min = [-10.97597381  -1.42974282  -2.29038963   9.25698967]


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [13]:
def calc_pred_proba(W,X):
    y_pred_proba = sigmoid(W.T@X)
    return y_pred_proba
    

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [30]:
def calc_pred(W,X,threshold=0.5):
    y_pred = sigmoid(W.T@X)>=threshold
    y_pred=y_pred.astype(np.float64)
    return y_pred
    

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [31]:
W,err = eval_model(X_st, y, iterations=9500, alpha=1e-1)
y_pred=calc_pred(W,X_st)
print(W)
print(err)

[-10.97597381  -1.42974282  -2.29038963   9.25698967]
0.2554502533028652


In [32]:
#В качестве меры точности возьмем долю правильных ответов
accuracy = 100.0 - np.mean(np.abs(y_pred - y)*100.0)
print(accuracy)

90.0


In [33]:
print(y)
print(y_pred)

[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]
[0. 0. 1. 0. 1. 0. 1. 0. 0. 1.]


In [44]:
#матрицa ошибок
def errors_matrix(y,y_pred):
    TP=0
    FP=0
    TN=0
    FN=0
    for i in np.arange(y.shape[0]):
        if y[i]==1 and y_pred[i]==1:
            TP+=1
        elif y[i]==0 and y_pred[i]==0:
            TN+=1
        elif y[i]==0 and y_pred[i]==1:
            FP+=1
        elif y[i]==1 and y_pred[i]==0:
            FN+=1
    
    matrix=np.array([[TP,FP],[FN,TN]])
    return matrix    



In [45]:
matrix=errors_matrix(y,y_pred)
print(matrix)

[[4 0]
 [1 5]]


точность 
$$precision(a, X) = \frac{TP}{TP+FP}.$$

In [47]:
precision=matrix[0,0]/(matrix[0,0]+matrix[0,1])
print(precision)

1.0


полнота
$$recall(a, X) = \frac{TP}{TP+FN},$$

In [48]:
recall=matrix[0,0]/(matrix[0,0]+matrix[1,0])
print(recall)

0.8


$$F_{\beta} = (1 + \beta^{2}) \frac{precision \cdot recall}{\beta^{2} \cdot precision + recall}.$$

In [49]:
F1=2*precision*recall/(precision+recall)
print(F1)

0.888888888888889


6. Могла ли модель переобучиться? Почему?

Да ,  могла, т.к. 
1. Использовалась одна маленькая выборка, не использовались Кросс-валидация или метод отложенной выборки
2. Одним из знаков, что произошло переобучение модели, или мерой сложности является получение больших по модулю весов при признаках

Необходимо поэтому использовать Кросс-валидация или метод отложенной выборки, а также регуляризацию

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.


In [87]:
W,err = eval_model(X_st, y, iterations=10000, alpha=1e-2)
print(W)
print(err)

[-2.77079473 -0.99580928  0.56650766  3.2676589 ]
0.4058305387773311


In [88]:
def eval_model_l1(X, y, iterations, alpha=1e-4,lambda_=1e-4):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T)+ lambda_*np.sign(W))
    #if i % (iterations / 10) == 0:
     # print(i, W, err)
  return W,err

In [89]:
W,err = eval_model_l1(X_st, y, iterations=10000, alpha=1e-2)
print(W)
print(err)

[-2.76518516 -0.99406451  0.5655746   3.26155449]
0.40603260837253974


In [90]:
def eval_model_l2(X, y, iterations, alpha=1e-4,lambda_=1e-4):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T)+ lambda_ * W)
    #if i % (iterations / 10) == 0:
     # print(i, W, err)
  return W,err

In [91]:
W,err = eval_model_l2(X_st, y, iterations=10000, alpha=1e-2)
print(W)
print(err)

[-2.75739282 -0.99310968  0.56731166  3.25521524]
0.4063086113262118
